# **Importamos librerias**

In [ ]:
! pip install git+https://github.com/maciejkula/spotlight.git

  Cloning https://github.com/maciejkula/spotlight.git to /tmp/pip-req-build-w8hygukq
  Running command git clone --filter=blob:none --quiet https://github.com/maciejkula/spotlight.git /tmp/pip-req-build-w8hygukq
  Resolved https://github.com/maciejkula/spotlight.git to commit 75f4c8c55090771b52b88ef1a00f75bb39f9f2a9
  Preparing metadata (setup.py) ... done


In [ ]:


import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
import joblib
import torch
from spotlight.cross_validation import random_train_test_split
from spotlight.cross_validation import user_based_train_test_split
from spotlight.evaluation import precision_recall_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.interactions import Interactions

# **Extraemos la data**

In [ ]:
# conectamos al drive
drive.mount('/content/drive')

# Definimos el directorio
dir_data='./drive/MyDrive/alicorp/data'

# extraemos la data

df_user=pd.read_csv(filepath_or_buffer=dir_data+'/df_user.csv')

# extraemos los parametros

params=joblib.load(dir_data+'/alicorp_params_x0.sav')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Construimos las interaciones**

In [ ]:
# creamos una instancia de datasets

user_array=np.array(df_user['customer_id'])

product_array=np.array(df_user['product_id'])


interaction_matrix=Interactions(user_ids=user_array,
                                item_ids=product_array)



# **Picamos la data**

In [ ]:
(train,test)=random_train_test_split( interactions=interaction_matrix,
                                        test_percentage=0.1)

# **Creamos y entrenamos el modelo**

In [ ]:



model=ImplicitFactorizationModel(loss='adaptive_hinge',
                                 embedding_dim=params['embedding_dim'],
                                 n_iter=params['n_iter'],
                                 batch_size=params['batch_size'],
                                 l2=params['l2'],
                                 learning_rate=params['learning_rate'],
                                 num_negative_samples=params['num_negative_samples'],
                                 use_cuda=True)

model.fit(interactions=train,
          verbose=True)

Epoch 0: loss 0.7475366086219385
Epoch 1: loss 0.6158888270021587
Epoch 2: loss 0.5701539693741804
Epoch 3: loss 0.541103835404799
Epoch 4: loss 0.5184608445911749
Epoch 5: loss 0.5010721453127097
Epoch 6: loss 0.48816927882705796
Epoch 7: loss 0.47543462439819184
Epoch 8: loss 0.46651152394336093
Epoch 9: loss 0.4574144605806491


# **Evaluamos el modelo**

In [ ]:
recall_k30=precision_recall_score(model, test,k=30)[1]
recall_k50=precision_recall_score(model, test,k=50)[1]

In [ ]:
print(f'recall={np.round(recall_k30.mean(),4)}')
print(f'recall={np.round(recall_k50.mean(),4)}')

recall=0.7722
recall=0.8394


# **Exportamos el modelo**

In [ ]:
torch.save(model,dir_data+'/model.sav')